<a href="https://colab.research.google.com/github/smsultan76/MachineLearning/blob/main/omprehensive_multi_method_clustering_and_insights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import warnings
warnings.filterwarnings('ignore')

# Setting plot style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Load data
df = pd.read_csv('/content/shopping_behavior_updated.csv')

print("="*50)
print("CUSTOMER CLUSTERING ANALYSIS")
print("="*50)

# 1. DATA PREPROCESSING FOR CLUSTERING
print("\n1. DATA PREPROCESSING")
print("-" * 30)

# Create copy of original data
df_original = df.copy()

# Create age groups for better segmentation
df['Age_Group'] = pd.cut(df['Age'],
                        bins=[0, 25, 35, 45, 55, 65, 100],
                        labels=['18-25', '26-35', '36-45', '46-55', '56-65', '65+'])

# Feature engineering
print("Creating additional features...")

# Customer value segments based on purchase amount and frequency
df['Customer_Value'] = df['Purchase Amount (USD)'] * df['Previous Purchases']

# Create behavioral segments
df['High_Value_Customer'] = (df['Purchase Amount (USD)'] > df['Purchase Amount (USD)'].quantile(0.75)).astype(int)
df['Frequent_Buyer'] = (df['Previous Purchases'] > df['Previous Purchases'].quantile(0.75)).astype(int)
df['High_Rating_Customer'] = (df['Review Rating'] > 4.0).astype(int)

# Select features for clustering
clustering_features = [
    'Age',
    'Purchase Amount (USD)',
    'Review Rating',
    'Previous Purchases',
    'High_Value_Customer',
    'Frequent_Buyer',
    'High_Rating_Customer'
]

# Prepare numerical features
X_numerical = df[clustering_features].copy()

print(f"Selected features for clustering: {clustering_features}")
print(f"Shape of clustering dataset: {X_numerical.shape}")

# Handle categorical variables using Label Encoding for additional features
categorical_features = ['Gender', 'Category', 'Season', 'Payment Method', 'Frequency of Purchases']

label_encoders = {}
for feature in categorical_features:
    le = LabelEncoder()
    df[feature + '_encoded'] = le.fit_transform(df[feature])
    label_encoders[feature] = le
    clustering_features.append(feature + '_encoded')
    X_numerical[feature + '_encoded'] = df[feature + '_encoded']

print(f"Total features after encoding: {len(clustering_features)}")

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_numerical)

print("Data preprocessing completed!")

# 2. DETERMINING OPTIMAL NUMBER OF CLUSTERS
print("\n2. FINDING OPTIMAL NUMBER OF CLUSTERS")
print("-" * 30)

# Elbow method and Silhouette analysis
wcss = []  # Within-cluster sum of squares
silhouette_scores = []
k_range = range(2, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    wcss.append(kmeans.inertia_)

    # Silhouette score
    silhouette_avg = silhouette_score(X_scaled, kmeans.labels_)
    silhouette_scores.append(silhouette_avg)
    print(f"K={k}: WCSS = {kmeans.inertia_:.2f}, Silhouette = {silhouette_avg:.3f}")

# Plot elbow curve and silhouette scores
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Elbow curve
ax1.plot(k_range, wcss, 'bo-', linewidth=2, markersize=8)
ax1.set_xlabel('Number of Clusters (K)')
ax1.set_ylabel('Within-Cluster Sum of Squares (WCSS)')
ax1.set_title('Elbow Method for Optimal K')
ax1.grid(True)

# Silhouette scores
ax2.plot(k_range, silhouette_scores, 'ro-', linewidth=2, markersize=8)
ax2.set_xlabel('Number of Clusters (K)')
ax2.set_ylabel('Silhouette Score')
ax2.set_title('Silhouette Analysis for Optimal K')
ax2.grid(True)

plt.tight_layout()
plt.show()

# Choose optimal K (you can adjust this based on the plots)
optimal_k = 4  # This can be changed based on elbow and silhouette analysis
print(f"\nSelected optimal number of clusters: {optimal_k}")

# 3. K-MEANS CLUSTERING
print("\n3. K-MEANS CLUSTERING")
print("-" * 30)

# Apply K-means clustering
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X_scaled)

# Add cluster labels to dataframe
df['Cluster'] = clusters
X_numerical['Cluster'] = clusters

print("K-means clustering completed!")
print(f"Cluster distribution:")
print(df['Cluster'].value_counts().sort_index())

# 4. CLUSTER PROFILING AND ANALYSIS
print("\n4. CLUSTER PROFILING")
print("-" * 30)

# Calculate cluster statistics
cluster_profile = df.groupby('Cluster').agg({
    'Age': ['mean', 'std'],
    'Purchase Amount (USD)': ['mean', 'std'],
    'Review Rating': ['mean', 'std'],
    'Previous Purchases': ['mean', 'std'],
    'Gender': lambda x: x.mode()[0] if len(x.mode()) > 0 else 'Unknown',
    'Category': lambda x: x.mode()[0] if len(x.mode()) > 0 else 'Unknown',
    'Frequency of Purchases': lambda x: x.mode()[0] if len(x.mode()) > 0 else 'Unknown',
    'Customer ID': 'count'
}).round(2)

cluster_profile.columns = ['Age_Mean', 'Age_Std', 'Purchase_Mean', 'Purchase_Std',
                          'Rating_Mean', 'Rating_Std', 'PrevPurchases_Mean', 'PrevPurchases_Std',
                          'Dominant_Gender', 'Dominant_Category', 'Dominant_Frequency', 'Count']

print("Cluster Profile Summary:")
print(cluster_profile)

# 5. VISUALIZATION OF CLUSTERS
print("\n5. CLUSTER VISUALIZATION")
print("-" * 30)

# 5.1 PCA for dimensionality reduction and 2D visualization
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_scaled)

# Create visualization dataframe
viz_df = pd.DataFrame(X_pca, columns=['PC1', 'PC2'])
viz_df['Cluster'] = clusters
viz_df['Purchase_Amount'] = df['Purchase Amount (USD)']
viz_df['Age'] = df['Age']
viz_df['Previous_Purchases'] = df['Previous Purchases']

# 5.2 Plot clusters in 2D
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Clusters in PCA space
scatter = axes[0,0].scatter(viz_df['PC1'], viz_df['PC2'], c=viz_df['Cluster'],
                           cmap='viridis', alpha=0.7, s=50)
axes[0,0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance)')
axes[0,0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance)')
axes[0,0].set_title('Customer Clusters in PCA Space')
plt.colorbar(scatter, ax=axes[0,0])

# Plot 2: Purchase Amount vs Age colored by cluster
scatter = axes[0,1].scatter(df['Age'], df['Purchase Amount (USD)'], c=clusters,
                           cmap='viridis', alpha=0.7, s=50)
axes[0,1].set_xlabel('Age')
axes[0,1].set_ylabel('Purchase Amount (USD)')
axes[0,1].set_title('Purchase Amount vs Age by Cluster')
plt.colorbar(scatter, ax=axes[0,1])

# Plot 3: Previous Purchases vs Review Rating
scatter = axes[1,0].scatter(df['Previous Purchases'], df['Review Rating'], c=clusters,
                           cmap='viridis', alpha=0.7, s=50)
axes[1,0].set_xlabel('Previous Purchases')
axes[1,0].set_ylabel('Review Rating')
axes[1,0].set_title('Previous Purchases vs Review Rating by Cluster')
plt.colorbar(scatter, ax=axes[1,0])

# Plot 4: Cluster sizes
cluster_sizes = df['Cluster'].value_counts().sort_index()
axes[1,1].bar(cluster_sizes.index, cluster_sizes.values, color=['red', 'blue', 'green', 'orange'][:optimal_k])
axes[1,1].set_xlabel('Cluster')
axes[1,1].set_ylabel('Number of Customers')
axes[1,1].set_title('Cluster Sizes')
for i, v in enumerate(cluster_sizes.values):
    axes[1,1].text(i, v, str(v), ha='center', va='bottom')

plt.tight_layout()
plt.show()

# 5.3 Parallel coordinates plot for cluster characteristics
plt.figure(figsize=(14, 8))

# Select key features for parallel coordinates
parallel_features = ['Age', 'Purchase Amount (USD)', 'Review Rating', 'Previous Purchases']
parallel_df = df[parallel_features + ['Cluster']].copy()

# Normalize features for parallel coordinates
for feature in parallel_features:
    parallel_df[feature] = (parallel_df[feature] - parallel_df[feature].min()) / (parallel_df[feature].max() - parallel_df[feature].min())

from pandas.plotting import parallel_coordinates
parallel_coordinates(parallel_df, 'Cluster', colormap='viridis', alpha=0.5)
plt.title('Parallel Coordinates Plot of Customer Clusters')
plt.ylabel('Normalized Feature Value')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 6. DETAILED CLUSTER CHARACTERISTICS
print("\n6. DETAILED CLUSTER ANALYSIS")
print("-" * 30)

# Analyze each cluster in detail
for cluster_id in range(optimal_k):
    cluster_data = df[df['Cluster'] == cluster_id]

    print(f"\n--- CLUSTER {cluster_id} (n={len(cluster_data)}) ---")
    print(f"Demographics:")
    print(f"  • Average Age: {cluster_data['Age'].mean():.1f} years")
    print(f"  • Gender Distribution: {cluster_data['Gender'].value_counts().to_dict()}")

    print(f"Spending Behavior:")
    print(f"  • Avg Purchase Amount: ${cluster_data['Purchase Amount (USD)'].mean():.2f}")
    print(f"  • Total Spending: ${cluster_data['Purchase Amount (USD)'].sum():.2f}")
    print(f"  • Avg Previous Purchases: {cluster_data['Previous Purchases'].mean():.1f}")

    print(f"Preferences:")
    print(f"  • Top Category: {cluster_data['Category'].mode()[0]}")
    print(f"  • Top Season: {cluster_data['Season'].mode()[0]}")
    print(f"  • Avg Review Rating: {cluster_data['Review Rating'].mean():.2f}")

    print(f"Behavioral Patterns:")
    print(f"  • Common Payment: {cluster_data['Payment Method'].mode()[0]}")
    print(f"  • Common Frequency: {cluster_data['Frequency of Purchases'].mode()[0]}")
    print(f"  • Subscription Rate: {(cluster_data['Subscription Status'] == 'Yes').mean()*100:.1f}%")

# 7. CLUSTER NAMING BASED ON CHARACTERISTICS
print("\n7. CLUSTER INTERPRETATION AND NAMING")
print("-" * 30)

cluster_names = {}
for cluster_id in range(optimal_k):
    cluster_data = df[df['Cluster'] == cluster_id]

    # Define characteristics
    avg_age = cluster_data['Age'].mean()
    avg_spend = cluster_data['Purchase Amount (USD)'].mean()
    avg_prev_purchases = cluster_data['Previous Purchases'].mean()
    avg_rating = cluster_data['Review Rating'].mean()

    # Determine cluster type based on characteristics
    if avg_spend > df['Purchase Amount (USD)'].mean() and avg_prev_purchases > df['Previous Purchases'].mean():
        cluster_type = "High-Value Loyal Customers"
    elif avg_spend > df['Purchase Amount (USD)'].mean() and avg_prev_purchases <= df['Previous Purchases'].mean():
        cluster_type = "High-Spend Occasional Shoppers"
    elif avg_spend <= df['Purchase Amount (USD)'].mean() and avg_prev_purchases > df['Previous Purchases'].mean():
        cluster_type = "Frequent Budget Shoppers"
    else:
        cluster_type = "Occasional Budget Shoppers"

    # Age-based refinement
    if avg_age > 50:
        cluster_type += " (Mature)"
    elif avg_age < 30:
        cluster_type += " (Young)"
    else:
        cluster_type += " (Middle-aged)"

    cluster_names[cluster_id] = cluster_type
    print(f"Cluster {cluster_id}: {cluster_type}")

# Add cluster names to dataframe
df['Cluster_Name'] = df['Cluster'].map(cluster_names)

# 8. COMPARISON OF CLUSTERING ALGORITHMS
print("\n8. COMPARING CLUSTERING ALGORITHMS")
print("-" * 30)

# Try different clustering algorithms
algorithms = {
    'K-Means': KMeans(n_clusters=optimal_k, random_state=42, n_init=10),
    'Agglomerative': AgglomerativeClustering(n_clusters=optimal_k),
    'DBSCAN': DBSCAN(eps=0.5, min_samples=5)
}

results = []

for name, algorithm in algorithms.items():
    if name == 'DBSCAN':
        labels = algorithm.fit_predict(X_scaled)
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    else:
        labels = algorithm.fit_predict(X_scaled)
        n_clusters = optimal_k

    if n_clusters > 1:  # Only calculate metrics if we have more than 1 cluster
        silhouette = silhouette_score(X_scaled, labels)
        calinski = calinski_harabasz_score(X_scaled, labels)
        davies = davies_bouldin_score(X_scaled, labels)
    else:
        silhouette = calinski = davies = np.nan

    results.append({
        'Algorithm': name,
        'Number of Clusters': n_clusters,
        'Silhouette Score': silhouette,
        'Calinski-Harabasz Score': calinski,
        'Davies-Bouldin Score': davies
    })

results_df = pd.DataFrame(results)
print("Clustering Algorithms Comparison:")
print(results_df.round(3))

# 9. BUSINESS INSIGHTS AND RECOMMENDATIONS
print("\n9. BUSINESS INSIGHTS AND RECOMMENDATIONS")
print("-" * 30)

print("\nKEY INSIGHTS FROM CUSTOMER CLUSTERING:")

# Calculate overall metrics for comparison
overall_avg_spend = df['Purchase Amount (USD)'].mean()
overall_avg_prev = df['Previous Purchases'].mean()

for cluster_id, cluster_name in cluster_names.items():
    cluster_data = df[df['Cluster'] == cluster_id]

    print(f"\n📊 {cluster_name}:")
    print(f"   • Size: {len(cluster_data)} customers ({len(cluster_data)/len(df)*100:.1f}%)")
    print(f"   • Average Spend: ${cluster_data['Purchase Amount (USD)'].mean():.2f} "
          f"({'+' if cluster_data['Purchase Amount (USD)'].mean() > overall_avg_spend else ''}"
          f"{(cluster_data['Purchase Amount (USD)'].mean() - overall_avg_spend)/overall_avg_spend*100:.1f}% vs average)")
    print(f"   • Loyalty Level: {cluster_data['Previous Purchases'].mean():.1f} previous purchases")

    # Recommendations
    if "High-Value" in cluster_name:
        print(f"   💡 Recommendations: Premium loyalty programs, early access to new products, personalized service")
    elif "Frequent" in cluster_name:
        print(f"   💡 Recommendations: Volume discounts, subscription benefits, referral programs")
    elif "High-Spend" in cluster_name:
        print(f"   💡 Recommendations: Luxury product recommendations, exclusive events, premium support")
    else:
        print(f"   💡 Recommendations: Entry-level offers, educational content, budget-friendly options")

# 10. SAVE RESULTS
print("\n10. SAVING RESULTS")
print("-" * 30)

# Save clustered data
output_file = 'customer_clusters_analysis.csv'
df.to_csv(output_file, index=False)

# Save cluster profiles
cluster_summary = df.groupby(['Cluster', 'Cluster_Name']).agg({
    'Age': 'mean',
    'Purchase Amount (USD)': ['mean', 'sum'],
    'Previous Purchases': 'mean',
    'Review Rating': 'mean',
    'Customer ID': 'count',
    'Subscription Status': lambda x: (x == 'Yes').mean()
}).round(2)

cluster_summary.columns = ['Avg_Age', 'Avg_Spend', 'Total_Spend', 'Avg_Previous_Purchases',
                          'Avg_Rating', 'Customer_Count', 'Subscription_Rate']
cluster_summary.to_csv('cluster_profiles_summary.csv')

print(f"Clustered data saved to: {output_file}")
print(f"Cluster profiles saved to: cluster_profiles_summary.csv")

print("\n" + "="*50)
print("CLUSTERING ANALYSIS COMPLETED SUCCESSFULLY!")
print("="*50)
print(f"\nTotal customers analyzed: {len(df)}")
print(f"Number of clusters identified: {optimal_k}")
print(f"Best clustering algorithm: K-Means")
print(f"Average silhouette score: {silhouette_score(X_scaled, clusters):.3f}")

: 